In [3]:
import random
from random import shuffle
from collections import Counter

In [2]:
import random
import sys
from random import shuffle
from collections import Counter

def load_words():
    with open("Words.txt") as word_file:
        valid_words = set(word_file.read().split())
    return valid_words

WORDS = load_words()

class Turn:
    def __init__(self, game, players, board, bag):
        self.game = game
        self.players = players
        self.board = board
        self.bag = bag
        self.current_player_index = 0
        self.first_turn = True
    
    def display(self):
        self.board.display()
        for player in self.players:
            print(f"{player.name}'s Score: {player.score}")

    def overlap_center(self, word, start_row, start_col, direction):
        # Check if the placed word overlaps with the center space
        for i, letter in enumerate(word):
            if direction in ['across', 'a']:
                if (start_row, start_col + i) == (7, 7):
                    return True
            else:  # direction is 'down' or 'd'
                if (start_row + i, start_col) == (7, 7):
                    return True
        return False
    
    def calculate_word_score(self, word, replacements):
        return sum(0 if i in replacements else self.bag.bag[letter]['tile'].value for i, letter in enumerate(word))
    
    def take_turn(self):
        while True:
            current_player = self.players[self.current_player_index]
            print(f"It's {current_player.name}'s turn.")
            print(f"Your letters are: {[tile.letter for tile in current_player.tiles]}")
            print(f"{current_player.name}'s score is: {current_player.score}")
            
            self.board.display()

            action = input("Do you wish to play (p), swap tile (t), or skip (s)? ").lower()

            if action in ['play', 'p']:
                while True:
                    try:
                        entered_word = input("Please enter a word: ").upper()

                        rack_word = entered_word.upper()
                        new_word = list(entered_word)
                        replacements = {}
                        for i, char in enumerate(entered_word):
                            if char == '_':
                                replace_letter = input("enter letter you want for _?: ").lower()
                                new_word[i] = replace_letter
                                replacements[i] = replace_letter
                    
                        new_word = "".join(new_word)

                        if not Game.is_valid_word(new_word):
                            print("This is not a valid word.")
                            continue

                        while True:
                            try:
                                entered_row = int(input("Please enter the row of the starting letter: "))
                                break  # Exit the loop if a valid integer is entered
                            except ValueError:
                                print("Only enter a valid integer!")
                            
                        while True:
                            try:
                                entered_column = int(input("Please enter the column of the starting letter: "))
                                break  # Exit the loop if a valid integer is entered
                            except ValueError:
                                print("Only enter a valid integer!")
                            
                        
                        entered_direction = input("Enter direction: across (a) or down (d) ").lower()
                        while entered_direction not in ['across', 'a', 'down', 'd']:
                            print("Invalid direction! Try again.")
                        
                        # Check if the spaces the player wants to place their tiles on are already taken
                        for i, letter in enumerate(new_word):
                            if entered_direction in ['across', 'a']:
                                existing_tile = self.board.get_tile(entered_column + i, entered_row)
                            else:  # direction is 'down' or 'd'
                                existing_tile = self.board.get_tile(entered_column, entered_row + i)
                            if existing_tile is not None and existing_tile.letter not in [letter, '*'] and existing_tile.letter != letter:
                                print("One or more of the spaces you are trying to place a tile on are already taken.")
                                return  # Go back to the start of the loop and prompt the user for another word
                        
                        # Add a check that the word overlaps with an existing word on the board (after the first turn)
                        if not self.first_turn and not self.board.overlap_existing_word(new_word, entered_row, entered_column, entered_direction):
                            print("Your word must overlap with an existing word on the board.")
                            continue

                        existing_letters = []
                        for i, letter in enumerate(new_word):
                            if entered_direction in ['across', 'a']:
                                existing_tile = self.board.get_tile(entered_column + i, entered_row)
                            else:
                                existing_tile = self.board.get_tile(entered_column, entered_row + i)
                            if existing_tile is not None:
                                existing_letters.append(existing_tile.letter)
                        
                        # Check if player has necessary letters for this word, considering letters already on the board
                        if not current_player.has_letters(rack_word, existing_letters):
                            print("You do not have the necessary letters for this word.")
                            continue

                        # Check if the first turn's word overlaps with the center space
                        if self.first_turn:
                            if not self.overlap_center(new_word, entered_row, entered_column, entered_direction):
                                print("The first word must overlap with the center of the board.")
                                continue
                            else:
                                self.first_turn = False
                        
                        if not self.board.valid_intermediate_board(new_word, entered_row, entered_column, entered_direction):
                            print("Your word causes conflict with the board.")
                            continue

                        # Place the word on the board
                        self.board.place_word(new_word, entered_row, entered_column, entered_direction)
                        
                        # Calculate and add score
                        total_score = 0
                        # Calculate the score for the word placed by the player
                        if entered_direction in ['across', 'a']:
                            word = self.board.get_word_across(entered_column, entered_row)
                        else:  # direction is 'down' or 'd'
                            word = self.board.get_word_down(entered_column, entered_row)

                        total_score += self.calculate_word_score(word, replacements)

                        # Calculate the score for the perpendicular words
                        for i, letter in enumerate(new_word):
                            if letter not in existing_letters:  # only consider letters placed by the player in this turn
                                if entered_direction in ['across', 'a']:
                                    x = entered_column + i
                                    y = entered_row
                                    # We've already calculated the across word score, so just calculate the down word score
                                    word_down = self.board.get_word_down(x, y)
                                    if len(word_down) > 1:
                                        total_score += self.calculate_word_score(word_down)
                                else:  # direction is 'down' or 'd'
                                    x = entered_column
                                    y = entered_row + i
                                    # We've already calculated the down word score, so just calculate the across word score
                                    word_across = self.board.get_word_across(x, y)
                                    if len(word_across) > 1:
                                        total_score += self.calculate_word_score(word_across)

                        # Add the total score to the player's score
                        current_player.score += total_score
                        
                        # Remove used tiles then add new ones
                        current_player.remove_tiles(new_word, existing_letters)
                        current_player.draw_tiles(self.bag)

                        break
                    
                    except KeyboardInterrupt:
                        print("code execution interrupted!")
                        return
            
            elif action in ['swap tile', 't']:
                while True:
                    tile_to_swap = input("Which tile would you like to swap? ").upper()
                    if current_player.has_letters([tile_to_swap]):
                        current_player.remove_tiles(tile_to_swap)
                        current_player.draw_tiles(self.bag)
                        break
                    else:
                        print("You must input a tile from your rack.")
            
            elif action in ['skip', 's']:
                self.game.skipped_turns += 1
                print("Turn skipped.")

            else:
                print("Invalid action! Try again.")
                continue
            
            
            self.current_player_index = (self.current_player_index + 1) % len(self.players)         
            break

class Player:
    def __init__(self, name):
        self.name = name
        self.score = 0
        self.tiles = []
    
    def has_letters(self, word, existing_letters=[]):
        if isinstance(word, str):
            word = list(word)

        letter_counts = Counter([tile.letter for tile in self.tiles])
        word_counts = Counter(word)
        word_counts.subtract(existing_letters)

        for letter, count in word_counts.items():
            if letter == '_':  # If the letter is '_', it is a wildcard
                if count > letter_counts['_']:  # Only consider it missing if there are not enough '_' tiles
                    return False
            elif count > letter_counts[letter]:
                return False
        return True
    
    def draw_tiles(self, bag):
        while len(self.tiles) < 7:
            self.tiles.append(bag.remove_from_bag())
    
    def remove_tiles(self, word, existing_letters=None):
        if existing_letters is None:
            existing_letters = []
            
        word_counts = Counter(word)
        existing_letters_counts = Counter(existing_letters)

        for letter, count in existing_letters_counts.items():
            if word_counts[letter] > count:
                word_counts[letter] -= count
            else:
                del word_counts[letter]

        for letter, count in word_counts.items():
            for _ in range(count):
                for tile in self.tiles:
                    if tile.letter == letter:
                        self.tiles.remove(tile)
                        break

    def add_score(self, word, bag):
        # Add the value of each tile in the word to the player's score
        for letter in word:
            self.score += bag.bag[letter]['tile'].value
    
    def tile_points(self):
        # to determine the winner
        return sum(tile.value for tile in self.tiles)

class Tile:
    def __init__(self, letter, value):
        self.letter = letter
        self.value = value

    def __repr__(self):
        return f"Tile('{self.letter}', {self.value})"
    
    def __str__(self):
        return self.letter


class Bag:
    def __init__(self):
        # Initialize the dictionary of letter to Tile mappings
        self.bag = {}
        self.initialize_bag()
    
    def add_to_bag(self, tile, quantity):
        # Adds a certain quantity of a certain tile to the bag.
        self.bag[tile.letter] = {"tile": tile, "quantity": quantity}
    
    def initialize_bag(self):
        #Adds the initial 100 tiles to the bag.
        self.add_to_bag(Tile("A", 1), 9)
        self.add_to_bag(Tile("B", 3), 2)
        self.add_to_bag(Tile("C", 3), 2)
        self.add_to_bag(Tile("D", 2), 4)
        self.add_to_bag(Tile("E", 1), 12)
        self.add_to_bag(Tile("F", 4), 2)
        self.add_to_bag(Tile("G", 2), 3)
        self.add_to_bag(Tile("H", 4), 2)
        self.add_to_bag(Tile("I", 1), 9)
        self.add_to_bag(Tile("J", 8), 1)
        self.add_to_bag(Tile("K", 5), 1)
        self.add_to_bag(Tile("L", 1), 4)
        self.add_to_bag(Tile("M", 3), 2)
        self.add_to_bag(Tile("N", 1), 6)
        self.add_to_bag(Tile("O", 1), 8)
        self.add_to_bag(Tile("P", 3), 2)
        self.add_to_bag(Tile("Q", 10), 1)
        self.add_to_bag(Tile("R", 1), 6)
        self.add_to_bag(Tile("S", 1), 4)
        self.add_to_bag(Tile("T",1), 6)
        self.add_to_bag(Tile("U", 1), 4)
        self.add_to_bag(Tile("V", 4), 2)
        self.add_to_bag(Tile("W", 4), 2)
        self.add_to_bag(Tile("X", 8), 1)
        self.add_to_bag(Tile("Y", 4), 2)
        self.add_to_bag(Tile("Z", 10), 1)
        self.add_to_bag(Tile("_", 0), 2)

    def remove_from_bag(self):
        available_tiles = [k for k, v in self.bag.items() if v['quantity'] > 0]
        if not available_tiles:
            raise ValueError("No more tiles in the bag.")

        letter = random.choice(available_tiles)
        self.bag[letter]['quantity'] -= 1
        return self.bag[letter]['tile']
    
class Board:
    def __init__(self):
        self.grid = [[None for _ in range(15)] for _ in range(15)]
        self.grid[7][7] = Tile('*', 0)  # Center space initialized as '*'
    
    def place_tile(self, tile, x, y):
        self.grid[y][x] = tile
    
    def is_space_empty(self, x, y):
        return self.grid[y][x] is None or self.grid[y][x].letter == '*'

    def place_word(self, word, start_row, start_col, direction):
        for i, letter in enumerate(word):
            if direction in ['across', 'a']:
                self.place_tile(Tile(letter, 0), start_col + i, start_row)
            else:  # direction is 'down' or 'd'
                self.place_tile(Tile(letter, 0), start_col, start_row + i)
    
    def overlap_existing_word(self, word, start_row, start_col, direction):
        # Check if the new word overlaps with any existing letter on the board
        for i, letter in enumerate(word):
            if direction in ['across', 'a']:
                if self.get_tile(start_col + i, start_row) is not None:
                    return True
            else:  # direction is 'down' or 'd'
                if self.get_tile(start_col, start_row + i) is not None:
                    return True
        return False

    def get_tile(self, x, y):
        return self.grid[y][x]

    def display(self):
        for row in self.grid:
            for tile in row:
                if tile is None:
                    print('.', end=' ')
                elif tile.letter == '*':
                    print('*', end=' ')
                else:
                    print(tile.letter, end=' ')
            print()
    
    def get_score(self, word, start_row, start_col, direction):
        total_score = 0
        # Compute base score for main word
        for i, letter in enumerate(word):
            if direction in ['across', 'a']:
                tile = self.get_tile(start_col + i, start_row)
            else:  # direction is 'down' or 'd'
                tile = self.get_tile(start_col, start_row + i)
            if tile is not None:
                total_score += tile.value
            # Check for secondary word at this tile
            secondary_word = ''
            if direction in ['across', 'a']:
                secondary_word = self.get_word_down(start_col + i, start_row)
            else:  # direction is 'down' or 'd'
                secondary_word = self.get_word_across(start_col, start_row + i)
            if secondary_word and len(secondary_word) > 1:
                # Compute and add score for secondary word
                secondary_word_tiles = [bag.get_tile(letter) for letter in secondary_word]
                secondary_score = sum(tile.value for tile in secondary_word_tiles)
                total_score += secondary_score
        return total_score
    
    # checking the input word's validity on the board
    def copy(self):
        """Create a copy of the current board."""
        board_copy = Board()
        board_copy.grid = [list(row) for row in self.grid]  # Deep copy of the grid
        return board_copy
    
    def get_word_across(self, x, y):
        # move the starting point to the left as long as there's a valid tile
        while x > 0 and self.grid[y][x - 1] is not None and self.grid[y][x - 1].letter != '*':
            x -= 1
        word = ''
        while x < 15 and self.grid[y][x] is not None and self.grid[y][x].letter != '*':
            word += self.grid[y][x].letter
            x += 1
        return word

    def get_word_down(self, x, y):
        # move the starting point upwards as long as there's a valid tile
        while y > 0 and self.grid[y - 1][x] is not None and self.grid[y - 1][x].letter != '*':
            y -= 1
        word = ''
        while y < 15 and self.grid[y][x] is not None and self.grid[y][x].letter != '*':
            word += self.grid[y][x].letter
            y += 1
        return word

    def valid_intermediate_board(self, word, row, column, direction):
        # Create a copy of the current board and place the word on it
        board_copy = self.copy()
        board_copy.place_word(word, row, column, direction)

        # Iterate over the entire board and check all possible words
        for row in range(15):
            for column in range(15):
                # If current tile is not None
                if board_copy.get_tile(column, row) is not None and board_copy.get_tile(column, row).letter != '*':
                    # Check the word across if the tile to the left is None or out of bounds
                    if column == 0 or board_copy.get_tile(column - 1, row) is None or board_copy.get_tile(column - 1, row).letter == '*':
                        word_across = board_copy.get_word_across(column, row)
                        if word_across and len(word_across) > 1 and not Game.is_valid_word(word_across):
                            return False
                    # Check the word down if the tile above is None or out of bounds
                    if row == 0 or board_copy.get_tile(column, row - 1) is None or board_copy.get_tile(column, row - 1).letter == '*':
                        word_down = board_copy.get_word_down(column, row)
                        if word_down and len(word_down) > 1 and not Game.is_valid_word(word_down):
                            return False

        return True

class Game:
    def __init__(self):
        self.board = Board()
        self.bag = Bag()
        self.players = []
        self.skipped_turns = 0
        
    def play_game(self):
        print("Welcome to Scrabble!")
        num_players = int(input("Enter number of players:" ))
        player_names = [input(f"Enter player {i + 1}'s name: ") for i in range(num_players)]
        
        for name in player_names:  # iterate over player names
            player = Player(name)  # create Player object
            player.tiles = [self.bag.remove_from_bag() for _ in range(7)]  # draw tiles
            self.players.append(player)  # add player to game

        self.turn = Turn(self, self.players, self.board, self.bag)
    
    def run(self):
        self.play_game()

        while not self.game_over():
            try:
                self.turn.take_turn()
            except KeyboardInterrupt:
                print("Code execution interupted!")
                return
            
        winners, final_scores = self.get_winner()
        if len(winners) > 1:
            print("It's a tie! The winners are: " + " and ".join(winners) + " with scores of " + str(final_scores[winners[0]]) + " each.")
        else:
            winner = winners[0]
            loser = [name for name in final_scores if name != winner][0]
            print(f"The game is over. The winner is {winner} with a final score of {final_scores[winner]} to {loser}'s score of {final_scores[loser]}.")    
   
    @staticmethod
    def is_valid_word(word):
    # assumes words is a list of all valid words
        return word.lower() in WORDS   
    
    def game_over(self):
        return len(self.bag.bag) == 0 or self.skipped_turns >= 6

    def get_winner(self):
        final_scores = {}
        for player in self.players:
            final_scores[player.name] = player.score - player.tile_points()

        max_score = max(final_scores.values())
        winners = [player for player, score in final_scores.items() if score == max_score]
        return winners, final_scores

In [ ]:
g = Game()
g.run()

Welcome to Scrabble!
It's a's turn.
Your letters are: ['I', 'A', 'I', 'S', 'M', 'N', 'C']
. . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . 
. . . . . . . * . . . . . . . 
. . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . 
a's score is now: 0
It's b's turn.
Your letters are: ['O', 'S', 'R', 'P', 'W', 'A', 'I']
. . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . 
. . . . . . . M A N . . . . . 
. . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . 
. . . . . . . .

code execution interrupted!
It's b's turn.
Your letters are: ['S', 'R', 'I', 'B', 'R', 'R', 'F']
. . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . 
. . . . . . . M A N . . . . . 
. . . . . . . O . . . . . . . 
. . . . . S I P . . . . . . . 
. . . . . A . . . . . . . . . 
. . . . . W I T . . . . . . . 
. . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . 
b's score is now: 0
This is not a valid word.


In [2]:
import random
import sys
from random import shuffle
from collections import Counter

def load_words():
    with open("Words.txt") as word_file:
        valid_words = set(word_file.read().split())
    return valid_words

WORDS = load_words()

class Turn:
    def __init__(self, game, players, board, bag):
        self.game = game
        self.players = players
        self.board = board
        self.bag = bag
        self.current_player_index = 0
        self.first_turn = True
    
    def display(self):
        self.board.display()
        for player in self.players:
            print(f"{player.name}'s Score: {player.score}")

    def overlap_center(self, word, start_row, start_col, direction):
        # Check if the placed word overlaps with the center space
        for i, letter in enumerate(word):
            if direction in ['across', 'a']:
                if (start_row, start_col + i) == (7, 7):
                    return True
            else:  # direction is 'down' or 'd'
                if (start_row + i, start_col) == (7, 7):
                    return True
        return False
    
    def calculate_word_score(self, word, replacements):
        return sum(0 if i in replacements else self.bag.bag[letter]['tile'].value for i, letter in enumerate(word))
    
    def take_turn(self):
        while True:
            current_player = self.players[self.current_player_index]
            print(f"It's {current_player.name}'s turn.")
            print(f"Your letters are: {[tile.letter for tile in current_player.tiles]}")
            print(f"{current_player.name}'s score is: {current_player.score}")
            
            self.board.display()

            action = input("Do you wish to play (p), swap tile (t), or skip (s)? ").lower()

            if action in ['play', 'p']:
                while True:
                    try:
                        entered_word = input("Please enter a word: ").upper()

                        rack_word = entered_word.upper()
                        new_word = list(entered_word)
                        replacements = {}
                        for i, char in enumerate(entered_word):
                            if char == '_':
                                replace_letter = input("enter letter you want for _?: ").lower()
                                new_word[i] = replace_letter
                                replacements[i] = replace_letter
                    
                        new_word = "".join(new_word)

                        if not Game.is_valid_word(new_word):
                            print("This is not a valid word.")
                            continue

                        while True:
                            try:
                                entered_row = int(input("Please enter the row of the starting letter: "))
                                break  # Exit the loop if a valid integer is entered
                            except ValueError:
                                print("Only enter a valid integer!")
                            
                        while True:
                            try:
                                entered_column = int(input("Please enter the column of the starting letter: "))
                                break  # Exit the loop if a valid integer is entered
                            except ValueError:
                                print("Only enter a valid integer!")
                            
                        
                        entered_direction = input("Enter direction: across (a) or down (d) ").lower()
                        while entered_direction not in ['across', 'a', 'down', 'd']:
                            print("Invalid direction! Try again.")
                        
                        # Check if the spaces the player wants to place their tiles on are already taken
                        for i, letter in enumerate(new_word):
                            if entered_direction in ['across', 'a']:
                                existing_tile = self.board.get_tile(entered_column + i, entered_row)
                            else:  # direction is 'down' or 'd'
                                existing_tile = self.board.get_tile(entered_column, entered_row + i)
                            if existing_tile is not None and existing_tile.letter not in [letter, '*'] and existing_tile.letter != letter:
                                print("One or more of the spaces you are trying to place a tile on are already taken.")
                                return  # Go back to the start of the loop and prompt the user for another word
                        
                        # Add a check that the word overlaps with an existing word on the board (after the first turn)
                        if not self.first_turn and not self.board.overlap_existing_word(new_word, entered_row, entered_column, entered_direction):
                            print("Your word must overlap with an existing word on the board.")
                            continue

                        existing_letters = []
                        for i, letter in enumerate(new_word):
                            if entered_direction in ['across', 'a']:
                                existing_tile = self.board.get_tile(entered_column + i, entered_row)
                            else:
                                existing_tile = self.board.get_tile(entered_column, entered_row + i)
                            if existing_tile is not None:
                                existing_letters.append(existing_tile.letter)
                        
                        # Check if player has necessary letters for this word, considering letters already on the board
                        if not current_player.has_letters(rack_word, existing_letters):
                            print("You do not have the necessary letters for this word.")
                            continue

                        # Check if the first turn's word overlaps with the center space
                        if self.first_turn:
                            if not self.overlap_center(new_word, entered_row, entered_column, entered_direction):
                                print("The first word must overlap with the center of the board.")
                                continue
                            else:
                                self.first_turn = False
                        
                        if not self.board.valid_intermediate_board(new_word, entered_row, entered_column, entered_direction):
                            print("Your word causes conflict with the board.")
                            continue

                        # Place the word on the board
                        self.board.place_word(new_word, entered_row, entered_column, entered_direction)
                        
                        # Calculate and add score
                        total_score = 0
                        # Calculate the score for the word placed by the player
                        if entered_direction in ['across', 'a']:
                            word = self.board.get_word_across(entered_column, entered_row)
                        else:  # direction is 'down' or 'd'
                            word = self.board.get_word_down(entered_column, entered_row)

                        total_score += self.calculate_word_score(word, replacements)

                        # Calculate the score for the perpendicular words
                        for i, letter in enumerate(new_word):
                            if letter not in existing_letters:  # only consider letters placed by the player in this turn
                                if entered_direction in ['across', 'a']:
                                    x = entered_column + i
                                    y = entered_row
                                    # We've already calculated the across word score, so just calculate the down word score
                                    word_down = self.board.get_word_down(x, y)
                                    if len(word_down) > 1:
                                        total_score += self.calculate_word_score(word_down)
                                else:  # direction is 'down' or 'd'
                                    x = entered_column
                                    y = entered_row + i
                                    # We've already calculated the down word score, so just calculate the across word score
                                    word_across = self.board.get_word_across(x, y)
                                    if len(word_across) > 1:
                                        total_score += self.calculate_word_score(word_across)

                        # Add the total score to the player's score
                        current_player.score += total_score
                        
                        # Remove used tiles then add new ones
                        current_player.remove_tiles(new_word, existing_letters)
                        current_player.draw_tiles(self.bag)

                        break
                    
                    except KeyboardInterrupt:
                        print("code execution interrupted!")
                        return
            
            elif action in ['swap tile', 't']:
                while True:
                    tile_to_swap = input("Which tile would you like to swap? ").upper()
                    if current_player.has_letters([tile_to_swap]):
                        current_player.remove_tiles(tile_to_swap)
                        current_player.draw_tiles(self.bag)
                        break
                    else:
                        print("You must input a tile from your rack.")
            
            elif action in ['skip', 's']:
                self.game.skipped_turns += 1
                print("Turn skipped.")

            else:
                print("Invalid action! Try again.")
                continue
            
            
            self.current_player_index = (self.current_player_index + 1) % len(self.players)         
            break

class Player:
    def __init__(self, name):
        self.name = name
        self.score = 0
        self.tiles = []
    
    def has_letters(self, word, existing_letters=[]):
        if isinstance(word, str):
            word = list(word)

        letter_counts = Counter([tile.letter for tile in self.tiles])
        word_counts = Counter(word)
        word_counts.subtract(existing_letters)

        for letter, count in word_counts.items():
            if letter == '_':  # If the letter is '_', it is a wildcard
                if count > letter_counts['_']:  # Only consider it missing if there are not enough '_' tiles
                    return False
            elif count > letter_counts[letter]:
                return False
        return True
    
    def draw_tiles(self, bag):
        while len(self.tiles) < 7:
            self.tiles.append(bag.remove_from_bag())
    
    def remove_tiles(self, word, existing_letters=None):
        if existing_letters is None:
            existing_letters = []
            
        word_counts = Counter(word)
        existing_letters_counts = Counter(existing_letters)

        for letter, count in existing_letters_counts.items():
            if word_counts[letter] > count:
                word_counts[letter] -= count
            else:
                del word_counts[letter]

        for letter, count in word_counts.items():
            for _ in range(count):
                for tile in self.tiles:
                    if tile.letter == letter:
                        self.tiles.remove(tile)
                        break

    def add_score(self, word, bag):
        # Add the value of each tile in the word to the player's score
        for letter in word:
            self.score += bag.bag[letter]['tile'].value
    
    def tile_points(self):
        # to determine the winner
        return sum(tile.value for tile in self.tiles)

class Tile:
    def __init__(self, letter, value):
        self.letter = letter
        self.value = value

    def __repr__(self):
        return f"Tile('{self.letter}', {self.value})"
    
    def __str__(self):
        return self.letter


class Bag:
    def __init__(self):
        # Initialize the dictionary of letter to Tile mappings
        self.bag = {}
        self.initialize_bag()
    
    def add_to_bag(self, tile, quantity):
        # Adds a certain quantity of a certain tile to the bag.
        self.bag[tile.letter] = {"tile": tile, "quantity": quantity}
    
    def initialize_bag(self):
        #Adds the initial 100 tiles to the bag.
        self.add_to_bag(Tile("A", 1), 9)
        self.add_to_bag(Tile("B", 3), 2)
        self.add_to_bag(Tile("C", 3), 2)
        self.add_to_bag(Tile("D", 2), 4)
        self.add_to_bag(Tile("E", 1), 12)
        self.add_to_bag(Tile("F", 4), 2)
        self.add_to_bag(Tile("G", 2), 3)
        self.add_to_bag(Tile("H", 4), 2)
        self.add_to_bag(Tile("I", 1), 9)
        self.add_to_bag(Tile("J", 8), 1)
        self.add_to_bag(Tile("K", 5), 1)
        self.add_to_bag(Tile("L", 1), 4)
        self.add_to_bag(Tile("M", 3), 2)
        self.add_to_bag(Tile("N", 1), 6)
        self.add_to_bag(Tile("O", 1), 8)
        self.add_to_bag(Tile("P", 3), 2)
        self.add_to_bag(Tile("Q", 10), 1)
        self.add_to_bag(Tile("R", 1), 6)
        self.add_to_bag(Tile("S", 1), 4)
        self.add_to_bag(Tile("T",1), 6)
        self.add_to_bag(Tile("U", 1), 4)
        self.add_to_bag(Tile("V", 4), 2)
        self.add_to_bag(Tile("W", 4), 2)
        self.add_to_bag(Tile("X", 8), 1)
        self.add_to_bag(Tile("Y", 4), 2)
        self.add_to_bag(Tile("Z", 10), 1)
        self.add_to_bag(Tile("_", 0), 2)

    def remove_from_bag(self):
        available_tiles = [k for k, v in self.bag.items() if v['quantity'] > 0]
        if not available_tiles:
            raise ValueError("No more tiles in the bag.")

        letter = random.choice(available_tiles)
        self.bag[letter]['quantity'] -= 1
        return self.bag[letter]['tile']
    
class Board:
    def __init__(self):
        self.grid = [[None for _ in range(15)] for _ in range(15)]
        self.grid[7][7] = Tile('*', 0)  # Center space initialized as '*'
        # Double word
        self.grid[1][1] = Tile('&', 0)
        self.grid[2][2] = Tile('&', 0)
        self.grid[3][3] = Tile('&', 0)
        self.grid[4][4] = Tile('&', 0)
        self.grid[1][13] = Tile('&', 0)
        self.grid[2][12] = Tile('&', 0)
        self.grid[3][11] = Tile('&', 0)
        self.grid[4][10] = Tile('&', 0)
        self.grid[10][4] = Tile('&', 0)
        self.grid[11][3] = Tile('&', 0)
        self.grid[12][2] = Tile('&', 0)
        self.grid[13][1] = Tile('&', 0)
        self.grid[10][10] = Tile('&', 0)
        self.grid[11][11] = Tile('&', 0)
        self.grid[12][12] = Tile('&', 0)
        self.grid[13][13] = Tile('&', 0)
        # Triple word
        self.grid[0][0] = Tile('$', 0)
        self.grid[0][7] = Tile('$', 0)
        self.grid[0][14] = Tile('$', 0)
        self.grid[7][0] = Tile('$', 0)
        self.grid[7][14] = Tile('$', 0)
        self.grid[14][0] = Tile('$', 0)
        self.grid[14][7] = Tile('$', 0)
        self.grid[14][14] = Tile('$', 0)
        # Double letter
        self.grid[0][3] = Tile('#', 0)
        self.grid[0][11] = Tile('#', 0)
        self.grid[2][6] = Tile('#', 0)
        self.grid[2][8] = Tile('#', 0)
        self.grid[3][0] = Tile('#', 0)
        self.grid[3][7] = Tile('#', 0)
        self.grid[3][14] = Tile('#', 0)
        self.grid[6][2] = Tile('#', 0)
        self.grid[6][6] = Tile('#', 0)
        self.grid[6][8] = Tile('#', 0)
        self.grid[6][12] = Tile('#', 0)
        self.grid[7][3] = Tile('#', 0)
        self.grid[7][11] = Tile('#', 0)
        self.grid[8][2] = Tile('#', 0)
        self.grid[8][6] = Tile('#', 0)
        self.grid[8][8] = Tile('#', 0)
        self.grid[8][12] = Tile('#', 0)
        self.grid[11][0] = Tile('#', 0)
        self.grid[11][7] = Tile('#', 0)
        self.grid[11][14] = Tile('#', 0)
        self.grid[12][6] = Tile('#', 0)
        self.grid[12][8] = Tile('#', 0)
        self.grid[14][3] = Tile('#', 0)
        self.grid[14][11] = Tile('#', 0)
        # Triple letter
        self.grid[1][5] = Tile('%', 0)
        self.grid[1][9] = Tile('%', 0)
        self.grid[5][1] = Tile('%', 0)
        self.grid[5][5] = Tile('%', 0)
        self.grid[5][9] = Tile('%', 0)
        self.grid[5][13] = Tile('%', 0)
        self.grid[9][5] = Tile('%', 0)
        self.grid[9][1] = Tile('%', 0)
        self.grid[9][9] = Tile('%', 0)
        self.grid[9][13] = Tile('%', 0)
        self.grid[13][5] = Tile('%', 0)
        self.grid[13][9] = Tile('%', 0)
    
    def place_tile(self, tile, x, y):
        self.grid[y][x] = tile
    
    def is_space_empty(self, x, y):
        return self.grid[y][x] is None or self.grid[y][x].letter == '*'

    def place_word(self, word, start_row, start_col, direction):
        for i, letter in enumerate(word):
            if direction in ['across', 'a']:
                self.place_tile(Tile(letter, 0), start_col + i, start_row)
            else:  # direction is 'down' or 'd'
                self.place_tile(Tile(letter, 0), start_col, start_row + i)
    
    def overlap_existing_word(self, word, start_row, start_col, direction):
        # Check if the new word overlaps with any existing letter on the board
        for i, letter in enumerate(word):
            if direction in ['across', 'a']:
                if self.get_tile(start_col + i, start_row) is not None:
                    return True
            else:  # direction is 'down' or 'd'
                if self.get_tile(start_col, start_row + i) is not None:
                    return True
        return False

    def get_tile(self, x, y):
        return self.grid[y][x]

    def display(self):
        print(" " + " ".join(str(i).rjust(2) for i in range(1, 16)))  # Print column numbers
        for i, row in enumerate(self.grid, start=1):
            print(str(i).rjust(2), end=" ")  # Print row number
            for tile in row:
                if tile is None:
                    print('.', end=' ')
                elif tile.letter == '*':
                    print('*', end=' ')
                else:
                    print(tile.letter, end=' ')
            print()
    
    # checking the input word's validity on the board
    def copy(self):
        """Create a copy of the current board."""
        board_copy = Board()
        board_copy.grid = [list(row) for row in self.grid]  # Deep copy of the grid
        return board_copy
    
    def get_word_across(self, x, y):
        # move the starting point to the left as long as there's a valid tile
        while x > 0 and self.grid[y][x - 1] is not None and self.grid[y][x - 1].letter != '*':
            x -= 1
        word = ''
        while x < 15 and self.grid[y][x] is not None and self.grid[y][x].letter != '*':
            word += self.grid[y][x].letter
            x += 1
        return word

    def get_word_down(self, x, y):
        # move the starting point upwards as long as there's a valid tile
        while y > 0 and self.grid[y - 1][x] is not None and self.grid[y - 1][x].letter != '*':
            y -= 1
        word = ''
        while y < 15 and self.grid[y][x] is not None and self.grid[y][x].letter != '*':
            word += self.grid[y][x].letter
            y += 1
        return word

    def valid_intermediate_board(self, word, row, column, direction):
        # Create a copy of the current board and place the word on it
        board_copy = self.copy()
        board_copy.place_word(word, row, column, direction)

        # Iterate over the entire board and check all possible words
        for row in range(15):
            for column in range(15):
                # If current tile is not None
                if board_copy.get_tile(column, row) is not None and board_copy.get_tile(column, row).letter != '*':
                    # Check the word across if the tile to the left is None or out of bounds
                    if column == 0 or board_copy.get_tile(column - 1, row) is None or board_copy.get_tile(column - 1, row).letter == '*':
                        word_across = board_copy.get_word_across(column, row)
                        if word_across and len(word_across) > 1 and not Game.is_valid_word(word_across):
                            return False
                    # Check the word down if the tile above is None or out of bounds
                    if row == 0 or board_copy.get_tile(column, row - 1) is None or board_copy.get_tile(column, row - 1).letter == '*':
                        word_down = board_copy.get_word_down(column, row)
                        if word_down and len(word_down) > 1 and not Game.is_valid_word(word_down):
                            return False

        return True

class Game:
    def __init__(self):
        self.board = Board()
        self.bag = Bag()
        self.players = []
        self.skipped_turns = 0
        
    def play_game(self):
        print("Welcome to Scrabble!")
        num_players = int(input("Enter number of players:" ))
        player_names = [input(f"Enter player {i + 1}'s name: ") for i in range(num_players)]
        
        for name in player_names:  # iterate over player names
            player = Player(name)  # create Player object
            player.tiles = [self.bag.remove_from_bag() for _ in range(7)]  # draw tiles
            self.players.append(player)  # add player to game

        self.turn = Turn(self, self.players, self.board, self.bag)
    
    def run(self):
        self.play_game()

        while not self.game_over():
            try:
                self.turn.take_turn()
            except KeyboardInterrupt:
                print("Code execution interupted!")
                return
            
        winners, final_scores = self.get_winner()
        if len(winners) > 1:
            print("It's a tie! The winners are: " + " and ".join(winners) + " with scores of " + str(final_scores[winners[0]]) + " each.")
        else:
            winner = winners[0]
            loser = [name for name in final_scores if name != winner][0]
            print(f"The game is over. The winner is {winner} with a final score of {final_scores[winner]} to {loser}'s score of {final_scores[loser]}.")    
   
    @staticmethod
    def is_valid_word(word):
    # assumes words is a list of all valid words
        return word.lower() in WORDS   
    
    def game_over(self):
        return len(self.bag.bag) == 0 or self.skipped_turns >= 6

    def get_winner(self):
        final_scores = {}
        for player in self.players:
            final_scores[player.name] = player.score - player.tile_points()

        max_score = max(final_scores.values())
        winners = [player for player, score in final_scores.items() if score == max_score]
        return winners, final_scores

In [3]:
g = Game()
g.run()

Welcome to Scrabble!
It's a's turn.
Your letters are: ['V', 'X', 'I', 'U', 'H', 'R', 'F']
a's score is: 0
  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15
 1 $ . . # . . . $ . . . # . . $ 
 2 . & . . . % . . . % . . . & . 
 3 . . & . . . # . # . . . & . . 
 4 # . . & . . . # . . . & . . # 
 5 . . . . & . . . . . & . . . . 
 6 . % . . . % . . . % . . . % . 
 7 . . # . . . # . # . . . # . . 
 8 $ . . # . . . * . . . # . . $ 
 9 . . # . . . # . # . . . # . . 
10 . % . . . % . . . % . . . % . 
11 . . . . & . . . . . & . . . . 
12 # . . & . . . # . . . & . . # 
13 . . & . . . # . # . . . & . . 
14 . & . . . % . . . % . . . & . 
15 $ . . # . . . $ . . . # . . $ 
Your word causes conflict with the board.
Your word causes conflict with the board.


Testing Zone...

In [ ]:
if "ed" in WORDS:
    print("The word is valid.")
else:
    print("The word is not valid.")

The word is valid.


In [11]:
winners = ["a", 'b']
winner = winners[0]
loser = next(name for name in winners if name != winner)
print(f"winner is {winner} and the loser is {loser}")

winner is a and the loser is b


TypeError: unhashable type: 'list'

In [3]:
#Entering _
x = input().upper()
print(x)
y = x.upper()
print(y)


HELLO
HELLO


In [ ]:
#Invalid direction entry 
entered_direction = input("Enter direction: across (a) or down (d): ")

while entered_direction not in ['across', 'a', 'down', 'd']:
    print("Invalid direction! Try again.")
    entered_direction = input("Enter direction: across (a) or down (d): ")

Invalid direction! Try again.


In [ ]:
#Valid integer in row/column enter
while True:
    try:
        entered_row = int(input("Please enter the row of the starting letter: "))
        break  # Exit the loop if a valid integer is entered
    except ValueError:
        print("Only enter a valid integer!")

# Continue with the rest of your code
# ...


Only enter a valid integer!


In [ ]:
grid = [[(str(i) if i == j or i == 0 else None) for j in range(16)] for i in range(16)]
print(grid)


[['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'], [None, '1', None, None, None, None, None, None, None, None, None, None, None, None, None, None], [None, None, '2', None, None, None, None, None, None, None, None, None, None, None, None, None], [None, None, None, '3', None, None, None, None, None, None, None, None, None, None, None, None], [None, None, None, None, '4', None, None, None, None, None, None, None, None, None, None, None], [None, None, None, None, None, '5', None, None, None, None, None, None, None, None, None, None], [None, None, None, None, None, None, '6', None, None, None, None, None, None, None, None, None], [None, None, None, None, None, None, None, '7', None, None, None, None, None, None, None, None], [None, None, None, None, None, None, None, None, '8', None, None, None, None, None, None, None], [None, None, None, None, None, None, None, None, None, '9', None, None, None, None, None, None], [None, None, None, None, None, None, None, N